# Gemini API: 安全快速指南


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/main/quickstarts/Safety.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


Gemini API 具有可調整的安全設定。本筆記本將逐步指導你如何使用這些設定。你將會撰寫一個遭到封鎖的提示，找出原因，然後調整篩選器以解鎖提示。

安全性是一個重要的主題，你可以前往本筆記本結尾處的連結了解更多資訊。本文著重於程式碼。


## 設定你的 API 金鑰

如果你想快速試用 Gemini API，可以使用 `curl` 指令呼叫 REST API 中的方法。

這個筆記本包含可在 Google Colab 中執行的 `curl` 指令，或複製到你的終端機。

要執行這個筆記本，你的 API 金鑰必須儲存在名為 GOOGLE_API_KEY 的 Colab 密碼中。如果你在不同的環境中執行，可以將你的金鑰儲存在環境變數中。請參閱 [驗證](https://github.com/doggy8088/gemini-api-cookbook/blob/main/quickstarts/Authentication.zh.ipynb) 以了解詳情。


In [ ]:
!apt install jq

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [108]:
os.environ['UNSAFE_PROMPT'] = 

## 提示回饋

[Model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 方法回傳的結果為 [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/generativeai/types/GenerateContentResponse)。


In [109]:
%%bash
echo '{
      "contents": [{
        "parts":[{
          "text": "'$UNSAFE_PROMPT'"}]}]}' > request.json

In [114]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$GOOGLE_API_KEY \
    -H 'Content-Type: application/json' \
    -X POST \
    -d @request.json  2> /dev/null | tee response.json

{
  "promptFeedback": {
    "blockReason": "SAFETY",
    "safetyRatings": [
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "probability": "NEGLIGIBLE"
      },
      {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "probability": "NEGLIGIBLE"
      },
      {
        "category": "HARM_CATEGORY_HARASSMENT",
        "probability": "MEDIUM"
      },
      {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "probability": "NEGLIGIBLE"
      }
    ]
  }
}


在上方你可以看到回應物件以兩種方式給你關於提示的安全回饋：

* `prompt_feedback.safety_ratings` 屬性包含一個列表，列出輸入提示的安全評分。
* 如果你的提示遭到封鎖，`prompt_feedback.block_reason` 欄位將會說明原因。


如果提示會因為安全評級而被封鎖，你在回應中不會取得任何候選人。


### 安全設定


調整安全設定和提示不再被封鎖：


In [134]:
%%bash
echo '{
    "safetySettings": [
        {'category': 7, 'threshold': 4}
    ],
    "contents": [{
        "parts":[{
          "text": "'$UNSAFE_PROMPT'"}]}]}' > request.json

In [143]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$GOOGLE_API_KEY \
    -H 'Content-Type: application/json' \
    -X POST \
    -d @request.json  2> /dev/null > response.json

有了新的設定，「blocked_reason」不再設定。


In [147]:
%%bash 

jq .promptFeedback < response.json

{
  "safetyRatings": [
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability": "NEGLIGIBLE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE"
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "MEDIUM"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE"
    }
  ]
}


並回傳一個候選回覆。


In [ ]:
%%bash 

jq .candidates[0].content.parts[].text < response.json

你可查看 `response.text` 以取得回應。


### 候選人評分


對於沒有被封鎖的提示，回應物件會包含一個 `候選` 物件清單 (現在只有 1 個)。每個候選都包含一個完成原因：


In [157]:
%%bash
jq .candidates[0].finishReason < response.json

"STOP"


`FinishReason.STOP` 指示模型在正常情況下完成其輸出。

`FinishReason.SAFETY` 指示候選者的 `safety_ratings` 超過請求的 `safety_settings` 閾值。


In [158]:
%%bash
jq .candidates[0].safetyRatings < response.json

[
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "probability": "NEGLIGIBLE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "probability": "NEGLIGIBLE"
  },
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "probability": "NEGLIGIBLE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "probability": "NEGLIGIBLE"
  }
]


## 進一步學習

閱讀這些關於 [安全指南](https://ai.google.dev/docs/safety_guidance) 和 [安全設定](https://ai.google.dev/docs/safety_setting_gemini) 的文章，深入了解。

## 有用的 API 參考

- 安全設定可以在 [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 建構函式中設定。它們也可以傳遞在每個 [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 或 [ChatSession.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession?hl=en#send_message) 要求中。
- [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) 會回傳 [SafetyRatings](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) 給 [GenerateContentResponse.prompt_feedback](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse/PromptFeedback) 中的提示，以及 `safety_ratings` 屬性中每個 [Candidate](https://ai.google.dev/api/python/google/ai/generativelanguage/Candidate)。
- [glm.SafetySetting](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetySetting) 包含： [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和 [glm.HarmBlockThreshold](https://ai.google.dev/api/python/google/generativeai/types/HarmBlockThreshold)
- [glm.SafetyRating](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) 包含 [HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和 [HarmProbability](https://ai.google.dev/api/python/google/generativeai/types/HarmProbability)
- [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 列舉值包含 PaLM 和 Gemini 型號的類別。Gemini 型號允許的值為 `[7,8,9,10]`: `[HARM_CATEGORY_HARASSMENT, HARM_CATEGORY_HATE_SPEECH, HARM_CATEGORY_SEXUALLY_EXPLICIT, HARM_CATEGORY_DANGEROUS_CONTENT]`.
- 在指定列舉值時，SDK 會接受列舉值本身，或它們的整數或字串表示形式。SKD 也會接受縮寫的字串表示形式：`["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"]` 都是有效的。字串不分大小寫。
